<a href="https://colab.research.google.com/github/pstorrer/Resumo_Noticias/blob/main/Resumo_Noticias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalar e Importar dependências



In [7]:
!pip install -U -q google-generativeai #gemini
!pip install feedparser #Parser para ler RSS Feeds.
import google.generativeai as genai #import gemini.
import feedparser #import Parser.
from google.colab import userdata as ud #import userdata para ler a akpi key do colab.
import pandas as pd #para criar o dataframe  com os modelos rss e salvar no histórico do model

# Carregar Parâmetros

In [4]:
GOOGLE_API_KEY = ud.get('API_GENAI')

# Configurações

In [5]:
genai.configure(api_key=GOOGLE_API_KEY)
listRSS = [] #global que irá salvar toda lista de RSS de resultados

# Funções

array_to_df função que irá ser utilizada para ler o list com todos os RSS pesquisados e adicionar no formato para importar no histórico.

In [ ]:
def array_to_df(array_of_strings):
  """
  Converte um array de strings em um DataFrame no formato adequado para o Model API.
  """
  # Criar uma lista de dicionários, assumindo que o usuário é o autor de todas as mensagens
  data = [{"role": "user", "content": message} for message in array_of_strings]

  # Criar o DataFrame
  df = pd.DataFrame(data)

  return df

scrap_google_news_feed função que recebe um termo de pesquisa e captura o RSS resultante do google News e salva no listRSS

In [ ]:
def scrape_google_news_feed(busca):
        rss_url = f'https://news.google.com/rss/search?q={busca}&hl=pt-BR&gl=BR&ceid=BR:pt-419'
        feed = feedparser.parse(rss_url)
        if feed.entries:
            for entry in feed.entries:
              listRSS.append(entry)

# Modelos

Modelo que gera os termos de pesquisa.

In [9]:
system_instruction = "Gerar um JSON de duas partes. A primeira parte um array de strings em forma de tópicos de pesquisa de notícias, a segunda parte o tempo máximo no exemplo última hora, última semana, últimos 30 dias. Caso o tempo não seja informado no texto do usuário, suponha 1 semana. Caso o texto do usuário seja muito genérico, por exemplo tudo da última hora, Liste estes 10 principais tópicos Política,Economia,Tecnologia,Saúde,Meio Ambiente,Educação,Cultura e Entretenimento,Esportes,Crime e Justiça,Comunidade Local para os arrays e '1 hora' para o tempo máximo, se tiver algum filtro de localidade incluir nos termos. A resposta gerada só pode ser tópicos de pesquisa, caso o texto do usuário não possa ser convertido em tópicos de pesquisa retorne o seguinte JSON: {\"-1\", \"O texto não gerou termos para pesquisa de notícias, reformule\"}\n"

modelo_termos = genai.GenerativeModel(model_name="gemini-1.5-pro-latest", system_instruction=system_instruction)

Modelo que resume as noticias de acordo com cada termo de pesquisa.

# Nova seção